<a href="https://colab.research.google.com/github/Shreya-07/CMPE255_catchup-assignment-part-1-/blob/main/DecisionTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Libraries

In [1]:
import pandas as pd
import numpy as np

In [27]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Loading Dataset

In [28]:
data = pd.read_csv('/content/gdrive/MyDrive/dataset_auth.txt',
                   names = ['variance','skewness', 'kurtosis', 'entropy', 'class'])
data


,variance,skewness,kurtosis,entropy,class
0,3.62160,8.66610,-2.8073,-0.44699,0
1,4.54590,8.16740,-2.4586,-1.46210,0
2,3.86600,-2.63830,1.9242,0.10645,0
3,3.45660,9.52280,-4.0112,-3.59440,0
4,0.32924,-4.45520,4.5718,-0.98880,0
...,...,...,...,...,...
1367,0.40614,1.34920,-1.4501,-0.55949,1
1368,-1.38870,-4.87730,6.4774,0.34179,1
1369,-3.75030,-13.45860,17.5932,-2.77710,1
1370,-3.56370,-8.38270,12.3930,-1.28230,1


In [4]:
X = data.drop('class', axis=1).to_numpy()
y = data['class'].to_numpy()

Defining Gini Impurity index

In [5]:
def gini_imp(y):
    y = np.asarray(y)
    m = y.shape[0]
    classes = np.unique(y)
    g_i = np.asarray([(np.sum(y == c)/m) for c in classes])
    G = 1 - np.sum(g_i**2)
    return G

In [6]:
def fetch_gini(G_left, G_right):
    m_l = G_left.size
    m_r = G_right.size
    G = (m_l/(m_l+m_r))*G_left + (m_r/(m_l+m_r))*G_right
    return G

In [7]:
def split_node(X, y, feature, threshold):
    X_left, y_left, X_right, y_right = [], [], [], []
    for (row,y_i) in zip(X,y):
        if row[feature] < threshold:
            X_left.append(row)
            y_left.append(y_i)
        else:
            X_right.append(row)
            y_right.append(y_i)
    return X_left, y_left, X_right, y_right

In [8]:
def best_split(X, y):
    X = np.asarray(X)
    y = np.asarray(y)
    best_feature, best_threshold, best_G, best_children = 0, 0, 2, 0
    for feature in range(X.shape[1]):
        for row in X:
            X_left, y_left, X_right, y_right = split_node(X, y, feature, row[feature])
            G_left  = gini_imp(y_left)
            G_right = gini_imp(y_right)
            G = fetch_gini(G_left, G_right)
            if G < best_G:
                Node_left = [X_left, y_left]
                Node_right = [X_right, y_right]
                best_feature, best_threshold, best_G, best_children = \
                feature, row[feature], G, [Node_left, Node_right]
    return dict({'feature':best_feature, 'threshold':best_threshold, 'children':best_children})

In [9]:
Xd = np.asarray([[2.771244718,1.784783929],
                [1.728571309,1.169761413],
                [3.678319846,2.81281357],
                [3.961043357,2.61995032],
                [2.999208922,2.209014212],
                [7.497545867,3.162953546],
                [9.00220326,3.339047188],
                [7.444542326,0.476683375],
                [10.12493903,3.234550982],
                [6.642287351,3.319983761]])
yd = np.asarray([0,0,0,0,0,1,1,1,1,1])

Building tree

In [10]:
def leaf(Node):
    X, y = Node
    return np.argmax(np.bincount(y))

In [11]:
def tree(Node, max_depth, min_size, depth): 
    
    left, right = Node['children']
    del(Node['children'])
    if not left or not right:
        Node['left'] = Node['right'] = leaf(left + right)
        return
    if depth >= max_depth:
        Node['left'], Node['right'] = leaf(left), leaf(right)
        return
    if len(left[0]) <= min_size:
        Node['left'] = leaf(left)
    else:
        Node['left'] = best_split(left[0], left[1])
        tree(Node['left'], max_depth, min_size, depth+1)
    if len(right[1]) <= min_size:
        Node['right'] = leaf(right)
    else:
        Node['right'] = best_split(right[0], right[1])
        tree(Node['right'], max_depth, min_size, depth+1)

In [12]:
def build_tree(X, y, max_depth, min_size):
    root = best_split(X, y)
    tree(root, max_depth, min_size, 1)
    return root

In [13]:
def print_tree(d, indent=0):
    for key, value in d.items():
        print('\t' * indent + str(key))
        if isinstance(value, dict):
            print_tree(value, indent+1)
        else:
            print('\t' * (indent+1) + str(value))

In [14]:
def predict(Node, row):
    if row[Node['feature']] < Node['threshold']:
        if isinstance(Node['left'], dict):
            return predict(Node['left'], row)
        else:
            return Node['left']
    else:
        if isinstance(Node['right'], dict):
            return predict(Node['right'], row)
        else:
            return Node['right']

In [15]:
def cart(X_train, y_train, X_test, y_test, max_depth, min_size):
    tree = build_tree(X_train, y_train, max_depth, min_size)
    predictions = []
    for row in X_test:
        prediction = predict(tree, row)
        predictions.append(prediction)
    return(predictions)

In [16]:
print_tree(build_tree(Xd, yd, 2, 1))

feature
	0
threshold
	6.642287351
left
	feature
		0
	threshold
		2.771244718
	left
		0
	right
		0
right
	feature
		0
	threshold
		7.497545867
	left
		1
	right
		1


In [21]:
pred = cart(Xd, yd, Xd, yd, 1, 1)
print(np.sum(pred==yd)/yd.shape[0])

1.0


In [18]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [20]:
y_prediction = cart(X_train, y_train, X_test, y_test, 5, 10)
print(confusion_matrix(y_test, y_prediction))
print(f1_score(y_test, y_prediction))

[[126  58]
 [  1 158]]
0.8426666666666667
